# Source contributions

In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import netCDF4 as nc
from itertools import compress
import datetime as dt
import seaborn as sns; sns.set()
import pandas as pd
from datetime import date

%matplotlib inline

###### Parameters:

In [2]:
imin, imax = 1480, 2180
jmin, jmax = 160, 800
isize = imax-imin
jsize = jmax-jmin

# model result set:
folder_ref      = '/data/brogalla/run_storage/Mn-reference-202103/'
folder_cleanice = '/data/brogalla/run_storage/Mn-clean-ice-202103/'
folder_spm      = '/data/brogalla/run_storage/Mn-spm-202103/'

colors = ['#ccb598', '#448d90', '#739f78', '#CC8741', '#cee7fd', '#b9c1c7']

In [3]:
years = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, \
         2017, 2018, 2019]

###### Load files:

In [4]:
mask      = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
tmask     = np.array(mask.variables['tmask'])[0,:,imin:imax,jmin:jmax]
land_mask = np.ma.masked_where((tmask[:,:,:] > 0.1), tmask[:,:,:]) 
e1t_base  = np.array(mask.variables['e1t'])[0,imin:imax,jmin:jmax]
e2t_base  = np.array(mask.variables['e2t'])[0,imin:imax,jmin:jmax]
e3t       = np.array(mask.variables['e3t_0'])[0,:,imin:imax,jmin:jmax]
e3t_masked = np.ma.masked_where((tmask[:,:,:] < 0.1), e3t)

e1t       = np.tile(e1t_base, (50,1,1))
e2t       = np.tile(e2t_base, (50,1,1))
volume    = e1t*e2t*e3t
area_base = e1t_base*e2t_base
volume_masked = np.ma.masked_where((tmask[:,:,:] < 0.1), volume)
area_masked   = np.ma.masked_where((tmask[0,:,:] < 0.1), area_base)

mesh      = nc.Dataset('/ocean/brogalla/GEOTRACES/ariane_runs/ANHA12-EXH006_y2015m01d05_gridT.nc')
lons      = np.array(mesh.variables['nav_lon'])
lats      = np.array(mesh.variables['nav_lat'])

##### Functions:

In [5]:
def load_results(folder_ref, year, experiment):
    
    months  = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    dmn_riv = np.empty((12,isize,jsize))   ; dmn_sed = np.empty((12,isize,jsize)); 
    dmn_atm = np.empty((12,isize,jsize))   ; dmn_ice = np.empty((12,isize,jsize));
    dmn_sedice = np.empty((12,isize,jsize));
    dmn_red = np.empty((12,50,isize,jsize)); dmn_oxi = np.empty((12,50,isize,jsize));
    dmn_priv = np.empty((12,isize,jsize))   ;
    
    for i, month in enumerate(months):
        file        = f'ANHA12_EXH006_1m_{year}0101_{year}1231_comp_{year}{month}-{year}{month}.nc'
        ref_monthly = nc.Dataset(folder_ref+file)
        
        dmn_riv[i,:,:]    = np.array(ref_monthly.variables['dmnriv'])[0,:,:]
        
        if experiment=='spm':
            dmn_priv[i,:,:] = np.array(ref_monthly.variables['pmnriv'])[0,:,:]
        else:
            dmn_priv[i,:,:] = 0
        
        dmn_sed[i,:,:]    = np.array(ref_monthly.variables['dmnsed'])[0,:,:]
        dmn_sedice[i,:,:] = np.array(ref_monthly.variables['dmnsedice'])[0,:,:]
        dmn_atm[i,:,:]    = np.array(ref_monthly.variables['dmnatm'])[0,:,:]
        dmn_ice[i,:,:]    = np.array(ref_monthly.variables['dmnice'])[0,:,:]
        dmn_red[i,:,:,:]  = np.array(ref_monthly.variables['dmnred'])[0,:,:,:]
        dmn_oxi[i,:,:,:]  = np.array(ref_monthly.variables['dmnoxi'])[0,:,:,:]
    
    tmask_surf    = np.empty(dmn_riv.shape)
    tmask_surf[:] = tmask[0,:,:]
    tmask_full    = np.empty(dmn_red.shape)
    tmask_full[:] = tmask
    
    # Mask points on land:
    dmn_mriv    = np.ma.masked_where((tmask_surf < 0.1), dmn_riv); 
    dmn_mpriv   = np.ma.masked_where((tmask_surf < 0.1), dmn_priv); 
    dmn_msed    = np.ma.masked_where((tmask_surf < 0.1), dmn_sed)
    dmn_msedice = np.ma.masked_where((tmask_surf < 0.1), dmn_sedice)
    dmn_matm    = np.ma.masked_where((tmask_surf < 0.1), dmn_atm)
    dmn_mice    = np.ma.masked_where((tmask_surf < 0.1), dmn_ice)
    dmn_mred    = np.ma.masked_where((tmask_full < 0.1), dmn_red)
    dmn_moxi    = np.ma.masked_where((tmask_full < 0.1), dmn_oxi)

    return dmn_mriv, dmn_mpriv, dmn_msed, dmn_msedice, dmn_matm, dmn_mice, dmn_mred , dmn_moxi

In [6]:
def calculate_contribution(year, mask, experiment='ref'):
    # Calculate the contribution of the model components by region:
    
    if experiment=='spm':
        folder_year = folder_spm + f'ref-{year}-spm-202103/'
    else:
        folder_year = folder_ref + f'ref-{year}-202103/'
    
    dmn_mriv, dmn_mpriv, dmn_msed, dmn_msedice, dmn_matm, dmn_mice, dmn_mred, dmn_moxi = \
                    load_results(folder_year, year, experiment)
    
    # Calculate contributions to the upper water column, so mask locations where sediment resuspension is added in the deep.
    indexes_bottom = np.tile(index_bottom, (12,1,1))
    dmn_msed = np.ma.masked_where(indexes_bottom > 19, dmn_msed)
    
    # moles per second
    priv   = np.ma.masked_where(mask==0, dmn_mpriv)   * volume_masked[0,:,:] * 1e3
    riv    = np.ma.masked_where(mask==0, dmn_mriv)    * volume_masked[0,:,:] * 1e3
    sed    = np.ma.masked_where(mask==0, dmn_msed)    * volume_bottom * 1e3
    sedice = np.ma.masked_where(mask==0, dmn_msedice) * volume_masked[0,:,:] * 1e3
    atm    = np.ma.masked_where(mask==0, dmn_matm)    * volume_masked[0,:,:] * 1e3
    ice    = np.ma.masked_where(mask==0, dmn_mice)    * volume_masked[0,:,:] * 1e3
    
#     # moles per meter squared per second
#     privm2   = np.ma.masked_where(mask==0, dmn_mpriv)   * e3t_masked[0,:,:] * 1e3
#     rivm2    = np.ma.masked_where(mask==0, dmn_mriv)    * e3t_masked[0,:,:] * 1e3
#     sedm2    = np.ma.masked_where(mask==0, dmn_msed)    * e3t_bottom * 1e3
#     sedicem2 = np.ma.masked_where(mask==0, dmn_msedice) * e3t_masked[0,:,:] * 1e3
#     atmm2    = np.ma.masked_where(mask==0, dmn_matm)    * e3t_masked[0,:,:] * 1e3
#     icem2    = np.ma.masked_where(mask==0, dmn_mice)    * e3t_masked[0,:,:] * 1e3
    
    # Yearly contribution to domain: moles / year
    atm_year    = np.ma.sum(atm[:,:,:], axis=(0,1,2))   *3600*24*365 / 12
    riv_year    = np.ma.sum(riv[:,:,:], axis=(0,1,2))   *3600*24*365 / 12
    priv_year   = np.ma.sum(priv[:,:,:], axis=(0,1,2))  *3600*24*365 / 12
    sed_year    = np.ma.sum(sed[:,:,:], axis=(0,1,2))   *3600*24*365 / 12
    sedice_year = np.ma.sum(sedice[:,:,:], axis=(0,1,2))*3600*24*365 / 12
    ice_year    = np.ma.sum(ice[:,:,:], axis=(0,1,2))   *3600*24*365 / 12
    
    # Average yearly contribution over domain: moles / m2 / year
    total_area     = np.ma.sum(np.ma.masked_where(mask[0,:,:]==0, area_masked[:,:]))
    atmm2_year     =  atm_year    / total_area
    rivm2_year     =  riv_year    / total_area
    privm2_year    =  priv_year   / total_area
    sedm2_year     =  sed_year    / total_area 
    sedicem2_year  =  sedice_year / total_area
    icem2_year     =  ice_year    / total_area
    
#     atmm2_year    = np.ma.average(np.ma.sum(atmm2[:,:,:], axis=(0)))   *3600*24*365
#     rivm2_year    = np.ma.average(np.ma.sum(rivm2[:,:,:], axis=(0)))   *3600*24*365
#     privm2_year   = np.ma.average(np.ma.sum(privm2[:,:,:], axis=(0)))  *3600*24*365
#     sedm2_year    = np.ma.average(np.ma.sum(sedm2[:,:,:], axis=(0)))   *3600*24*365
#     sedicem2_year = np.ma.average(np.ma.sum(sedicem2[:,:,:], axis=(0)))*3600*24*365
#     icem2_year    = np.ma.average(np.ma.sum(icem2[:,:,:], axis=(0)))   *3600*24*365                                            
    
    total_yearly_contribution = np.array([priv_year, riv_year, sed_year, sedice_year, ice_year, atm_year])           # mol/yr
    ave_yearly_contribution   = np.array([privm2_year, rivm2_year, sedm2_year, sedicem2_year, icem2_year, \
                                          atmm2_year]) # mol/m2/yr
    
    return total_yearly_contribution, ave_yearly_contribution

In [7]:
def calculate_regional_contributions(mask):
    
    totals_ref   = np.empty((len(years),6)); totals_spm   = np.empty((len(years),6))
    averages_ref = np.empty((len(years),6)); averages_spm = np.empty((len(years),6))

    totals_ref[:]   = np.NaN; totals_spm[:]    = np.NaN;
    averages_ref[:] = np.NaN; averages_spm[:]  = np.NaN;

    for i, year in enumerate(years):
        total_ref_year, ave_ref_year           = calculate_contribution(year, mask, experiment='ref')
        total_spm_year, ave_spm_year           = calculate_contribution(year, mask, experiment='spm')

        totals_ref[i,:]        = total_ref_year 
        totals_spm[i,:]        = total_spm_year 
        averages_ref[i,:]      = ave_ref_year  
        averages_spm[i,:]      = ave_spm_year 
    
    return totals_ref, totals_spm, averages_ref, averages_spm

In [8]:
def reshape_arrays(array_in):
    fixed      = np.zeros((len(years),5))
    
    for i in range(0,5):
        if i==0: # Sum the particulate and dissolved river contributions
            fixed[:,i] = array_in[:,i]+array_in[:,i+1]
        else:
            fixed[:,i] = array_in[:,i+1]
    
    return fixed

In [9]:
def pipeline(totals_input_ref, totals_input_spm, averages_input_ref, averages_input_spm):
    
    regions = ['CB', 'CAA', 'wCAA', 'eCAA']
    
    totals_fixed_ref = totals_input_ref.copy(); averages_fixed_ref = averages_input_ref.copy();
    totals_fixed_spm = totals_input_spm.copy(); averages_fixed_spm = averages_input_spm.copy();
    
    # Reshape arrays so that particulate and dissolved river contributions are combined into one.
    for region in regions:

        totals_fixed_ref[region]   = reshape_arrays(totals_input_ref[region])
        totals_fixed_spm[region]   = reshape_arrays(totals_input_spm[region])
        averages_fixed_ref[region] = reshape_arrays(averages_input_ref[region])
        averages_fixed_spm[region] = reshape_arrays(averages_input_spm[region])
    
    return totals_fixed_ref, totals_fixed_spm, averages_fixed_ref, averages_fixed_spm

In [10]:
def print_interannual_averages(totals_ref, totals_spm, averages_ref, averages_spm, location='Full'):

    print('Interannual average values in ' + location)
    print('----------------------------------------------------------------')
    total_ref_annual = np.sum(totals_ref[location], axis=1)
    total_spm_annual = np.sum(totals_spm[location], axis=1)
    
    print(f"{'Total annual contribution of Mn [Mmol/yr]:':<55}" +
          f"{np.average(total_ref_annual)*1e-6:<5.0f} ({np.average(total_spm_annual)*1e-6:<5.0f})")
    print(f"{'-- River discharge ---':<40}" +
          f" {np.average(totals_ref[location][:,0])*1e-6:<8.2f} ({np.average(totals_spm[location][:,0])*1e-6:<8.2f})")
    print(f"{'-- Sediment resuspension ---':<40}" +
          f" {np.average(totals_ref[location][:,1])*1e-6:<8.2f} ({np.average(totals_spm[location][:,1])*1e-6:<8.2f})")
    print(f"{'-- Sediment released by sea ice ---':<40}" +
          f" {np.average(totals_ref[location][:,2])*1e-6:<8.2f} ({np.average(totals_spm[location][:,2])*1e-6:<8.2f})")
    print(f"{'-- Dust released by sea ice ---':<40}" +
          f" {np.average(totals_ref[location][:,3])*1e-6:<8.2f} ({np.average(totals_spm[location][:,3])*1e-6:<8.2f})")
    print(f"{'-- Dust deposition ---':<40}" +
          f" {np.average(totals_ref[location][:,4])*1e-6:<8.2f} ({np.average(totals_spm[location][:,4])*1e-6:<8.2f})") 

    tref = np.sum(totals_ref[location], axis=(0,1))
    tspm = np.sum(totals_spm[location], axis=(0,1))

    print('----------------------------------------------------------------')
    print('Total annual contribution of Mn [%]:') 
    print(f"{'-- River discharge ---':<40}" +
          f"{np.sum(totals_ref[location][:,0])*100/tref:<5.2f} ({np.sum(totals_spm[location][:,0])*100/tspm:<5.2f})")
    print(f"{'-- Sediment resuspension ---':<40}" +
          f"{np.sum(totals_ref[location][:,1])*100/tref:<5.2f} ({np.sum(totals_spm[location][:,1])*100/tspm:<5.2f})")
    print(f"{'-- Sediment released by sea ice ---':<40}" +
          f"{np.sum(totals_ref[location][:,2])*100/tref:<5.2f} ({np.sum(totals_spm[location][:,2])*100/tspm:<5.2f})")
    print(f"{'-- Dust released by sea ice ---':<40}" +
          f"{np.sum(totals_ref[location][:,3])*100/tref:<5.2f} ({np.sum(totals_spm[location][:,3])*100/tspm:<5.2f})")
    print(f"{'-- Dust deposition ---':<40}" +
          f"{np.sum(totals_ref[location][:,4])*100/tref:<5.2f} ({np.sum(totals_spm[location][:,4])*100/tspm:<5.2f})")

    print('----------------------------------------------------------------')
    print(f"{'Average annual contribution of Mn [micromol/m2/yr]:':<55}" +
          f"{np.average(np.sum(averages_ref[location], axis=1)*1e6):<5.0f}" +
          f"({np.average(np.sum(averages_spm[location], axis=1)*1e6):<5.0f})")
    print(f"{'-- River discharge ---':<40}" +
          f"{np.average(averages_ref[location][:,0])*1e6:<8.2f} ({np.average(averages_spm[location][:,0])*1e6:<8.2f})")
    print(f"{'-- Sediment resuspension ---':<40}" +
          f"{np.average(averages_ref[location][:,1])*1e6:<8.2f} ({np.average(averages_spm[location][:,1])*1e6:<8.2f})")
    print(f"{'-- Sediment released by sea ice ---':<40}" +
          f"{np.average(averages_ref[location][:,2])*1e6:<8.2f} ({np.average(averages_spm[location][:,2])*1e6:<8.2f})")
    print(f"{'-- Dust released by sea ice ---':<40}" +
          f"{np.average(averages_ref[location][:,3])*1e6:<8.2f} ({np.average(averages_spm[location][:,3])*1e6:<8.2f})")
    print(f"{'-- Dust deposition ---':<40}" +
          f"{np.average(averages_ref[location][:,4])*1e6:<8.2f} ({np.average(averages_spm[location][:,4])*1e6:<8.2f})")

    return

### Calculations:

Find grid cell volume and thickness at ocean floor:

In [11]:
tmask_bot = np.copy(tmask)
tmask_bot[0,:,:] = 1
e3t_mask = np.ma.masked_where((tmask_bot[:,:,:] < 0.1), e3t[:,:,:])

# find bottom grid cell index:
ind = np.array(np.ma.notmasked_edges(e3t_mask[:,:,:], axis=0))

# array of shape (isize, jsize) containing bottom grid cell index
index_bottom = np.ma.masked_where((tmask[0,:,:] < 0.1), ind[1][0][:].reshape(isize,jsize))
# index_bottom = np.ma.argmax(e3t_masked[:,:,:], axis=0)

In [12]:
e3t_bot = np.zeros((isize,jsize))
vol_bot = np.zeros((isize,jsize))

for i in range(0, isize):
    for j in range(0,jsize):
        k = index_bottom[i,j]
        try:
            e3t_bot[i,j] = e3t[k,i,j]
            vol_bot[i,j] = volume[k,i,j]
        except:
            e3t_bot[i,j] = np.nan
            vol_bot[i,j] = np.nan
        
e3t_bottom    = np.ma.masked_where((tmask[0,:,:] < 0.1), np.ma.masked_where(np.isnan(e3t_bot), e3t_bot))
volume_bottom = np.ma.masked_where((tmask[0,:,:] < 0.1), np.ma.masked_where(np.isnan(vol_bot), vol_bot))

Create masks for specific subregions:
- Canada Basin
- CAA
- Full domain
- Western CAA (separated by Barrow Sill)
- Eastern CAA (separated by Barrow Sill)

In [13]:
# Find indices to separate out Canada Basin:

x_ind = np.arange(1620, 2100, 1)
y_ind = (-7/8)*x_ind + 1517 + 700

CB_indx = []
CB_indy = []

for index in range(0,len(x_ind)):
    CB_x = np.arange(x_ind[index],2180,1)
    CB_y = np.ones(CB_x.shape)*y_ind[index]
    
    CB_indx = np.append(CB_x, CB_indx)
    CB_indy = np.append(CB_y, CB_indy)

In [14]:
# Separate Canada Basin and the CAA:
mask_ini_CB  = np.zeros((isize,jsize))
mask_ini_CAA = np.ones((isize,jsize))

for i, j in zip(CB_indx, CB_indy):
    mask_ini_CB[int(i-imin),int(j-jmin)]  = 1
    mask_ini_CAA[int(i-imin),int(j-jmin)] = 0
    
mask_ini_CB[150:-1 ,-8:-1] = 1
mask_ini_CAA[150:-1,-8:-1] = 0


In [15]:
mask_ini_wCAA = np.zeros((isize,jsize))
mask_ini_eCAA = np.zeros((isize,jsize))

# Separate the western and eastern CAA:
mask_ini_wCAA = np.where(lons[imin:imax,jmin:jmax] < -100, 1, 0)
mask_ini_eCAA = np.where(lons[imin:imax,jmin:jmax] > -100, 1, 0)

for i, j in zip(CB_indx, CB_indy):
    mask_ini_wCAA[int(i-imin),int(j-jmin)] = 0
    mask_ini_eCAA[int(i-imin),int(j-jmin)] = 0
    
mask_ini_wCAA[150:-1,-8:-1] = 0
mask_ini_eCAA[150:-1,-8:-1] = 0

In [16]:
mask_CB_yr    = np.tile(mask_ini_CB  , (12,1,1))
mask_CAA_yr   = np.tile(mask_ini_CAA , (12,1,1))
mask_full_yr  = np.ones((12, isize, jsize))
mask_wCAA_yr  = np.tile(mask_ini_wCAA, (12,1,1))
mask_eCAA_yr  = np.tile(mask_ini_eCAA, (12,1,1))

In [17]:
mask_CB_depth = np.tile(mask_CB_yr  , (50,1,1,1))
mask_CB_depth = mask_CB_depth.reshape(12,50,700,640)

### Calculate:

In [87]:
CB_totals_ref, CB_totals_spm, CB_averages_ref, CB_averages_spm = calculate_regional_contributions(mask_CB_yr)

In [88]:
CAA_totals_ref, CAA_totals_spm, CAA_averages_ref, CAA_averages_spm = calculate_regional_contributions(mask_CAA_yr)

In [89]:
wCAA_totals_ref, wCAA_totals_spm, wCAA_averages_ref, wCAA_averages_spm = calculate_regional_contributions(mask_wCAA_yr)

In [90]:
eCAA_totals_ref, eCAA_totals_spm, eCAA_averages_ref, eCAA_averages_spm = calculate_regional_contributions(mask_eCAA_yr)

### Pickle the results from these calculations:

In [91]:
pickle.dump((CB_totals_ref, CAA_totals_ref, wCAA_totals_ref, eCAA_totals_ref),\
            open('calculations/ref-total_component_contributions-20210418-fullareased-fulldepth.pickle','wb'))

In [92]:
pickle.dump((CB_totals_spm, CAA_totals_spm, wCAA_totals_spm, eCAA_totals_spm),\
            open('calculations/spm-total_component_contributions-20210418-fullareased-fulldepth.pickle','wb'))

In [93]:
pickle.dump((CB_averages_ref, CAA_averages_ref, wCAA_averages_ref, eCAA_averages_ref),\
            open('calculations/ref-averages_component_contributions-20210418-fullareased-fulldepth.pickle','wb'))

In [94]:
pickle.dump((CB_averages_spm, CAA_averages_spm, wCAA_averages_spm, eCAA_averages_spm),\
            open('calculations/spm-averages_component_contributions-20210418-fullareased-fulldepth.pickle','wb'))

### Overview:

In [18]:
CB_totals_ref, CAA_totals_ref, wCAA_totals_ref, eCAA_totals_ref = \
                pickle.load(open('calculations/ref-total_component_contributions-20210418-fullareased-fulldepth.pickle','rb')) 
CB_totals_spm, CAA_totals_spm, wCAA_totals_spm, eCAA_totals_spm = \
                pickle.load(open('calculations/spm-total_component_contributions-20210418-fullareased-fulldepth.pickle','rb')) 
CB_averages_ref, CAA_averages_ref, wCAA_averages_ref, eCAA_averages_ref = \
                pickle.load(open('calculations/ref-averages_component_contributions-20210418-fullareased-fulldepth.pickle','rb')) 
CB_averages_spm, CAA_averages_spm, wCAA_averages_spm, eCAA_averages_spm = \
                pickle.load(open('calculations/spm-averages_component_contributions-20210418-fullareased-fulldepth.pickle','rb')) 

In [19]:
ref_totals   = {'CB': CB_totals_ref, 'CAA':CAA_totals_ref, 'wCAA':wCAA_totals_ref, 'eCAA':eCAA_totals_ref}
spm_totals   = {'CB': CB_totals_spm, 'CAA':CAA_totals_spm, 'wCAA':wCAA_totals_spm, 'eCAA':eCAA_totals_spm}
ref_averages = {'CB': CB_averages_ref, 'CAA':CAA_averages_ref, 'wCAA':wCAA_averages_ref, 'eCAA':eCAA_averages_ref}
spm_averages = {'CB': CB_averages_spm, 'CAA':CAA_averages_spm, 'wCAA':wCAA_averages_spm, 'eCAA':eCAA_averages_spm}

In [20]:
final_ref_totals, final_spm_totals, \
    final_ref_averages, final_spm_averages = pipeline(ref_totals, spm_totals, ref_averages, spm_averages)

In [21]:
print_interannual_averages(final_ref_totals, final_spm_totals, final_ref_averages, final_spm_averages, location='CB')

Interannual average values in CB
----------------------------------------------------------------
Total annual contribution of Mn [Mmol/yr]:             385   (417  )
-- River discharge ---                   10.37    (42.17   )
-- Sediment resuspension ---             135.58   (135.58  )
-- Sediment released by sea ice ---      238.44   (238.44  )
-- Dust released by sea ice ---          0.39     (0.39    )
-- Dust deposition ---                   0.06     (0.06    )
----------------------------------------------------------------
Total annual contribution of Mn [%]:
-- River discharge ---                  2.69  (10.12)
-- Sediment resuspension ---            35.23 (32.54)
-- Sediment released by sea ice ---     61.96 (57.23)
-- Dust released by sea ice ---         0.10  (0.09 )
-- Dust deposition ---                  0.02  (0.01 )
----------------------------------------------------------------
Average annual contribution of Mn [micromol/m2/yr]:    193  (209  )
-- River discharge --- 

In [22]:
print_interannual_averages(final_ref_totals, final_spm_totals, final_ref_averages, final_spm_averages, location='CAA')

Interannual average values in CAA
----------------------------------------------------------------
Total annual contribution of Mn [Mmol/yr]:             1605  (1755 )
-- River discharge ---                   17.82    (168.21  )
-- Sediment resuspension ---             1534.06  (1534.06 )
-- Sediment released by sea ice ---      52.78    (52.78   )
-- Dust released by sea ice ---          0.31     (0.31    )
-- Dust deposition ---                   0.04     (0.04    )
----------------------------------------------------------------
Total annual contribution of Mn [%]:
-- River discharge ---                  1.11  (9.58 )
-- Sediment resuspension ---            95.58 (87.39)
-- Sediment released by sea ice ---     3.29  (3.01 )
-- Dust released by sea ice ---         0.02  (0.02 )
-- Dust deposition ---                  0.00  (0.00 )
----------------------------------------------------------------
Average annual contribution of Mn [micromol/m2/yr]:    1343 (1469 )
-- River discharge ---

In [23]:
print_interannual_averages(final_ref_totals, final_spm_totals, final_ref_averages, final_spm_averages, location='wCAA')

Interannual average values in wCAA
----------------------------------------------------------------
Total annual contribution of Mn [Mmol/yr]:             295   (304  )
-- River discharge ---                   2.94     (11.95   )
-- Sediment resuspension ---             267.75   (267.75  )
-- Sediment released by sea ice ---      24.52    (24.52   )
-- Dust released by sea ice ---          0.12     (0.12    )
-- Dust deposition ---                   0.02     (0.02    )
----------------------------------------------------------------
Total annual contribution of Mn [%]:
-- River discharge ---                  0.99  (3.93 )
-- Sediment resuspension ---            90.66 (87.97)
-- Sediment released by sea ice ---     8.30  (8.06 )
-- Dust released by sea ice ---         0.04  (0.04 )
-- Dust deposition ---                  0.01  (0.01 )
----------------------------------------------------------------
Average annual contribution of Mn [micromol/m2/yr]:    581  (599  )
-- River discharge --

In [24]:
print_interannual_averages(final_ref_totals, final_spm_totals, final_ref_averages, final_spm_averages, location='eCAA')

Interannual average values in eCAA
----------------------------------------------------------------
Total annual contribution of Mn [Mmol/yr]:             1310  (1451 )
-- River discharge ---                   14.88    (156.26  )
-- Sediment resuspension ---             1266.31  (1266.31 )
-- Sediment released by sea ice ---      28.26    (28.26   )
-- Dust released by sea ice ---          0.18     (0.18    )
-- Dust deposition ---                   0.02     (0.02    )
----------------------------------------------------------------
Total annual contribution of Mn [%]:
-- River discharge ---                  1.14  (10.77)
-- Sediment resuspension ---            96.69 (87.27)
-- Sediment released by sea ice ---     2.16  (1.95 )
-- Dust released by sea ice ---         0.01  (0.01 )
-- Dust deposition ---                  0.00  (0.00 )
----------------------------------------------------------------
Average annual contribution of Mn [micromol/m2/yr]:    1907 (2113 )
-- River discharge --

### Figures

Year-to-year variation in contribution

In [68]:
# sea_ice = '/ocean/brogalla/GEOTRACES/data/Sea_Ice_Index_Rates_of_Change_G02135_v3.0.csv'

# regions are defined in: 
# https://nsidc.org/sites/nsidc.org/files/files/data/noaa/g02135/Sea-Ice-Analysis-Spreadsheets-Overview.pdf
# This file is for the Beaufort Sea region, i.e. Canada Basin
sea_ice_CB = '/ocean/brogalla/GEOTRACES/data/N_Sea_Ice_Index_Regional_Monthly_Data_G02135_v3.0.csv'
sea_ice_CA = '/ocean/brogalla/GEOTRACES/data/N_Sea_Ice_Index_Regional_Monthly_Data_G02135_v3.0-central-arctic.csv'

si_CB = pd.read_csv(sea_ice_CB)
si_CA = pd.read_csv(sea_ice_CA)

In [ ]:
# observ_data = np.array(si[2:])

# observ_years       = observ_data[:-2,0].astype('float')
# monthly_ice_change = observ_data[:-2,1:].astype('float')

# monthly_ice_melt   = np.copy(monthly_ice_change)
# monthly_ice_melt[monthly_ice_change > 0] = 0 

# # Calculate the sum of all the melt in a year:
# yearly_melt = np.abs(np.sum(monthly_ice_melt, axis=1))

In [28]:
observ_data_CB = np.array(si_CB)
observ_data_CA = np.array(si_CA)

observ_years_CB     = observ_data_CB[:,0].astype('float')
observ_years_CA     = observ_data_CA[:,0].astype('float')
monthly_ice_area_CB = observ_data_CB[:,1:].astype('float')
monthly_ice_area_CA = observ_data_CA[:,1:].astype('float')

ice_areas_CB = monthly_ice_area_CB.flatten()
ice_areas_CA = monthly_ice_area_CA.flatten()
ice_areas_diff_CB = [ice_areas_CB[i+1] - ice_areas_CB[i] for i in range(0,len(ice_areas_CB)-1)]
ice_areas_diff_CA = [ice_areas_CA[i+1] - ice_areas_CA[i] for i in range(0,len(ice_areas_CA)-1)]

ice_areas_diff_CB = np.append(np.nan, ice_areas_diff_CB)
ice_areas_diff_CA = np.append(np.nan, ice_areas_diff_CA)

monthly_area_change_CB = ice_areas_diff_CB.reshape([41,12])
monthly_area_change_CA = ice_areas_diff_CA.reshape([41,12])
    
monthly_area_melt_CB = np.copy(monthly_area_change_CB)
monthly_area_melt_CA = np.copy(monthly_area_change_CA)
monthly_area_melt_CB[monthly_area_change_CB > 0] = 0 
monthly_area_melt_CA[monthly_area_change_CA > 0] = 0 

# Calculate the sum of all the melt in a year:
yearly_melt_CB = np.abs(np.sum(monthly_area_melt_CB, axis=1))
yearly_melt_CA = np.abs(np.sum(monthly_area_melt_CA, axis=1))

In [29]:
fig = plt.figure(figsize=(8,5))
st  = sns.axes_style("white")

with st:
#     sip = sns.lineplot(years, totals[:,2]*1e-6, linewidth=2.5, color=colors[5], marker="o", \
#                        markersize=7, label='Mn from sediment in sea ice [Mmol/yr]')    
#     osi = sns.lineplot(observ_years, yearly_melt*1e3, linewidth=2.5, marker='o', markersize=7, \
#                        label='Arctic sea ice extent reduction from satellite [1e3 Mkm2/yr]')
#     sip = sns.lineplot(years, totals_CB[:,2]*1e-6, linewidth=2.5, color=colors[5], marker="o", \
#                        markersize=7, label=r'Mn released from sea ice in Canada Basin [Mmol yr$^{-1}$]')
    sip = sns.lineplot(years, np.sum(totals_CB[:,:]*1e-6, axis=1), linewidth=2.5, color=colors[5], marker="o", \
                       markersize=7, label=r'External Mn addition to Canada Basin [Mmol yr$^{-1}$]')
    osi = sns.lineplot(observ_years_CB, yearly_melt_CB*1e-2, linewidth=2.5, marker='o', markersize=7, \
                       label=r'Sea ice melt in the Beaufort Sea from satellite [100 km$^{2}$ yr$^{-1}$]')

    sip.set_xlabel('Year', fontsize=14)
    sip.set_ylabel('Annual total',fontsize=14)
    sip.set(xlim=(2002, 2020), ylim=(0, 12000))
    sip.tick_params(axis='both', which='major', labelsize=14)
    sip.spines['left'].set_linewidth(1.5);   sip.spines['left'].set_color('black');
    sip.spines['bottom'].set_linewidth(1.5); sip.spines['bottom'].set_color('black');
    sip.spines['right'].set_linewidth(1.5);  sip.spines['right'].set_color('black');
    sip.spines['top'].set_linewidth(1.5);    sip.spines['top'].set_color('black');    
    
#     sip.ticklabel_format(style='scientific', axis='y')
#     osi.ticklabel_format(style='scientific', axis='y')
    
    sip.legend(loc=(0.05, 0.8), frameon=False)
    
# fig.savefig('/ocean/brogalla/GEOTRACES/figures/D1-annual-sea-ice-melt-'+ \
#             str(date.today().strftime('%Y%m%d'))+'.png', bbox_inches='tight',dpi=300)

NameError: name 'totals_CB' is not defined

<Figure size 576x360 with 0 Axes>

#### Canada Basin Mn content over time:

I'm curious whether a steady increase is visible. In that case it would be interesting to look at whether this is related to the increased freshwater storage of the Beaufort Gyre over the course of the 2000s.

In [ ]:
def read_yearly_mn(years):
    
    yearly_mn = []
    for year in years:
        Mn_model1  = nc.Dataset(results_folder+'ref-'+str(year)+'/ANHA12_EXH006_'+str(year)+'_monthly.nc')
        Mn_dis1    = np.array(Mn_model1.variables['dissolmn'])[:,0,:,:,:]
        
        Mn_CB      = np.ma.masked_where(mask_CB_depth==0, Mn_dis1)
        
        # Polar mixed layer:
        Mn_yearly  = np.ma.sum(Mn_CB[:,0:17,:,:], axis=0)
        Mn_content = np.ma.sum(Mn_yearly*volume_masked[0:17,:,:]*1e3, axis=(0,1,2))
        yearly_mn.append(Mn_content) # moles
        
    return np.array(yearly_mn)

In [ ]:
CB_mn_timeseries = read_yearly_mn(years)

In [ ]:
fig = plt.figure(figsize=(8,5))
st  = sns.axes_style("whitegrid")

with st:
    ax = sns.lineplot(observ_years_CB, yearly_melt_CB*1e-2, linewidth=2.5, marker='o', markersize=7, c='#46697b',\
                       label=r'Satellite sea ice loss in the Beaufort Sea [100 km$^{2}$ yr$^{-1}$]', zorder=1)
    sns.lineplot(years, totals_CB[:,2]*1e-6, linewidth=2.5, color='#a6b3c0', marker="o", \
                 markersize=7, label=r'Addition of Mn by sea ice melt in Canada Basin [Mmol yr$^{-1}$]', zorder=3)
    sns.lineplot(years, CB_mn_timeseries*1e-6, linewidth=2.5, marker='o', markersize=7, c='#6497b1', \
                 label=r'Mn content of the Canada Basin Polar Mixed Layer [Mmol]', zorder=2)
#     ax = sns.lineplot(years, np.sum(totals_CB[:,:]*1e-6, axis=1), linewidth=2.5, color=colors[5], marker="o", \
#                        markersize=7, label=r'External Mn addition to Canada Basin [Mmol yr$^{-1}$]')
    
    ax.set_xlabel('Year', fontsize=14)
    ax.set_ylabel('Annual total',fontsize=14)
    ax.set(xlim=(2002, 2020), ylim=(0, 1.2e4))
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.spines['left'].set_linewidth(1.5);   ax.spines['left'].set_color('black');
    ax.spines['bottom'].set_linewidth(1.5); ax.spines['bottom'].set_color('black');
    ax.spines['right'].set_linewidth(1.5);  ax.spines['right'].set_color('black');
    ax.spines['top'].set_linewidth(1.5);    ax.spines['top'].set_color('black');    
    
#     ax.ticklabel_format(style='scientific', axis='y')
#     osi.ticklabel_format(style='scientific', axis='y')
    
    leg = ax.legend(loc=(0.03, 0.76), frameon=True, framealpha=1, fontsize=12)
    leg.get_frame().set_linewidth(0.0)
    
fig.savefig('/ocean/brogalla/GEOTRACES/figures/D1-annual-sea-ice-melt-'+ \
            str(date.today().strftime('%Y%m%d'))+'.png', bbox_inches='tight',dpi=300)

Separated by region:

In [ ]:
fig = plt.figure(figsize=(7,5))
st  = sns.axes_style("white")

with st:
    rp  = sns.lineplot(years, totals_CB[:,0]*1e-6, linewidth=2.5, color=colors[1], label='Rivers')
    sp  = sns.lineplot(years, totals_CB[:,1]*1e-6, linewidth=2.5, color=colors[3], label='Sediment resuspension')
    sip = sns.lineplot(years, totals_CB[:,2]*1e-6, linewidth=2.5, color=colors[5], label='Sediment from sea ice')
    dip = sns.lineplot(years, totals_CB[:,3]*1e-6, linewidth=2.5, color=colors[4], label='Dust from sea ice')
    dp  = sns.lineplot(years, totals_CB[:,4]*1e-6, linewidth=2.5, color=colors[0], label='Dust deposition')

    rp2  = sns.lineplot(years, totals_CAA[:,0]*1e-6, linewidth=2.5, color=colors[1])
    sp2  = sns.lineplot(years, totals_CAA[:,1]*1e-6, linewidth=2.5, color=colors[3])
    sip2 = sns.lineplot(years, totals_CAA[:,2]*1e-6, linewidth=2.5, color=colors[5])
    dip2 = sns.lineplot(years, totals_CAA[:,3]*1e-6, linewidth=2.5, color=colors[4])
    dp2  = sns.lineplot(years, totals_CAA[:,4]*1e-6, linewidth=2.5, color=colors[0])

    rp.lines[5].set_linestyle("--")
    rp.lines[6].set_linestyle("--")
    rp.lines[7].set_linestyle("--")
    rp.lines[8].set_linestyle("--")
    rp.lines[9].set_linestyle("--")
    
    rp.set_xlabel('Year', fontsize=14)
    rp.set_ylabel('Mn contribution [Mmol/yr]',fontsize=14)
    rp.set(xlim=(2002, 2020))
    rp.tick_params(axis='both', which='major', labelsize=14)
    rp.spines['left'].set_linewidth(1.5);   rp.spines['left'].set_color('black');
    rp.spines['bottom'].set_linewidth(1.5); rp.spines['bottom'].set_color('black');
    rp.spines['right'].set_linewidth(1.5);  rp.spines['right'].set_color('black');
    rp.spines['top'].set_linewidth(1.5);    rp.spines['top'].set_color('black');    
    
# fig.savefig('/ocean/brogalla/GEOTRACES/figures/distr-PML_src.png', bbox_inches='tight',dpi=300)